# Di chuyển dữ liệu tư bronze qua silver


## 1. Kiểm tra dữ liệu

In [0]:
#dbutils.fs.rm("dbfs:/user/hive/warehouse/lakehouse.db/silver",recurse=True)

In [0]:
# %sql
# select * from bronze where id = 115408968

In [0]:
%sql
use lakehouse;

-- select legal_status from bronze
-- group by legal_status

In [0]:
%sql
-- select house_type from bronze
-- group by house_type

In [0]:
%sql
-- select priority_characteristics from bronze
-- group by priority_characteristics

In [0]:
%sql
-- select apartment_type from bronze
-- group by apartment_type

## 2. Extract, Transform and Load

In [0]:
from pyspark.sql.types import *
import re 

### 2.1 Đọc dữ liệu

In [0]:
# Đọc và bỏ giá trị trùng
df = spark.read.format("delta").table("bronze")
#df = df.dropDuplicates(["id"])

### 2.2 Sử lý các giá trị của các cột

In [0]:
# Sử lý cột giá và giá trên m2

# def parse_value(value):
#     if "." in value:
#         value = value.replace('.', '')
#     if "," in value:
#         value = value.replace(',', '.')  # Chuyển dấu phẩy thành dấu chấm
#     if "triệu" in value:
#         match = re.search(r'(\d+(\.\d+)?)\s*triệu', value)
#         if match:
#             numeric_value = float(match.group(1)) * 1000000
#             return float(numeric_value)
#     elif "tỷ" in value:
#         match = re.search(r'(\d+(\.\d+)?)\s*tỷ', value)
#         if match:
#             numeric_value = float(match.group(1)) * 1000000000
#             return float(numeric_value)
#     else:
#         #value = value.split()[0]
#         return value

# parse_value_udf = udf(parse_value,FloatType())
def parse_value(value):

  # Only remove periods if they are followed by a non-digit:
  if "." in value and not re.search(r'\d\.', value):
    value = value.replace('.', '')

  if "," in value:
    value = value.replace(',', '.')  # Convert comma to period (assuming decimal)

  if "triệu" in value:
    match = re.search(r'(\d+(\.\d+)?) triệu', value)
    if match:
      return float(match.group(1)) * 1000000

  elif "tỷ" in value:
    match = re.search(r'(\d+(\.\d+)?) tỷ', value)
    if match:
      return float(match.group(1)) * 1000000000

  else:
    # Try parsing the entire value as a number (assuming no units):
    try:
      return float(value)
    except ValueError:
      pass  # Return original value if parsing fails

  return value  # Return original value if no parsing is done or parsing fails

parse_value_udf = udf(parse_value, FloatType())
# Áp dụng UDF vào cột "value" và tạo một cột mới "parsed_value"
df = df.withColumn("price", parse_value_udf("price"))
#df = df.withColumn("price_per_square", parse_value_udf("price_per_square"))

# Hiển thị kết quả

In [0]:
df = df.filter(df.price >= 150000000)

In [0]:
# def parse_value(value):
#     if "m2" in value:
#         value = value.replace('m2','m²')
#   # Only remove periods if they are followed by a non-digit:
#     if "." in value and not re.search(r'\d\.', value):
#         value = value.replace('.', '')

#     if "," in value:
#         value = value.replace(',', '.')  # Convert comma to period (assuming decimal)
#     if "triệu/m²" in value:
#         match = re.search(r'(\d+(\.\d+)?) triệu', value)
#     if match:
#         return float(match.group(1)) * 1000000

#     elif "đ/m²" in value:
#         match = re.search(r'(\d+(\.\d+)?) tỷ', value)
#     if match:
#         return float(match.group(1)) 
#     else:
#     # Try parsing the entire value as a number (assuming no units):
#         try:
#             return float(value)
#         except ValueError:
#             pass  # Return original value if parsing fails

#     return value  # Return original value if no parsing is done or parsing fails

# parse_value_udf = udf(parse_value, FloatType())
# # Áp dụng UDF vào cột "value" và tạo một cột mới "parsed_value"
# df = df.withColumn("price_per_square", parse_value_udf("price_per_square"))

In [0]:
# xử lý cho cột địa chỉ
def xuly_diachi(value):
    if "Xem bản đồ" in value:
        value = value.replace('Xem bản đồ', '')

    value = ",".join(value.split(",")[-3:])
    return value

parse_value_udf = udf(xuly_diachi,StringType())
df = df.withColumn("address", parse_value_udf("address"))

In [0]:
# Sử lý cột area, length, width 

def xuly_donvi(value):
    if value is not None:
        value = value.split()[0]
        return float(value) 
    return None
parse_value_udf = udf(xuly_donvi,FloatType())
df = df.withColumn("area", parse_value_udf("area"))
df = df.withColumn("length", parse_value_udf("length"))
df = df.withColumn("width", parse_value_udf("width"))
df = df.withColumn("living_size", parse_value_udf("living_size"))



In [0]:
#df = df.select("price","area")

def re_calculate(price, area):
    value = price / area
    return round(value, 2)

parse_value_udf = udf(re_calculate,FloatType())
df = df.withColumn("price_per_square", parse_value_udf("price","area"))

In [0]:
# Sử lý cột toilet, bedroom

def xuly_phong(value):
    if value is not None:
        match = re.search(r'\d+', value)
        numbers = int(match.group())
        if "nhiều" in value or "Nhiều" in value:
            return numbers + 1
        else:
            return numbers 
    return 0
parse_value_udf = udf(xuly_phong,IntegerType())
df = df.withColumn("toilet", parse_value_udf("toilet"))
df = df.withColumn("bedroom", parse_value_udf("bedroom"))


In [0]:
%sql
-- select distinct priority_characteristics from bronze

In [0]:
# Thêm cột mới type
def bds_type(house_type,land_type,apartment_type):
    if house_type is not None:
        return "House"
    elif land_type is not None:
        return "Land"
    elif apartment_type is not None:
        return "Apartment"
    else:
        return "Else"
    
parse_value_udf = udf(bds_type, StringType())
df = df.withColumn("type", parse_value_udf(df["house_type"],df["land_type"],df["apartment_type"]))


In [0]:
# Chỉnh sửa cột floor

def bds_floor(type,floor):
    if floor is not None:
        return int(floor)
    elif "house" in type:
        return 0
    else:
        return 0
    
parse_value_udf = udf(bds_floor, IntegerType())
df = df.withColumn("floor", parse_value_udf(df["type"],df["floor"]))




In [0]:
# Thêm cột mặt tiền
def land_frontage(house_type, priority_characteristics):
    if house_type is not None and "mặt tiền" in house_type:
        return 1
    if priority_characteristics is not None and "Mặt tiền" in priority_characteristics:
        return 1
    else: 
        return 0 
parse_value_udf = udf(land_frontage, IntegerType())
df = df.withColumn("land_frontage", parse_value_udf(df["house_type"],df["priority_characteristics"]))

# nở hậu
def spacious_backyard(priority_characteristics):
    if priority_characteristics is not None and "nở hậu" in priority_characteristics:
        return 1
    else: 
        return 0 
parse_value_udf = udf(spacious_backyard, IntegerType())
df = df.withColumn("spacious_backyard", parse_value_udf(df["priority_characteristics"]))

#hem o to
def drive_way(priority_characteristics):
    if priority_characteristics is not None and "Hẻm xe hơi" in priority_characteristics:
        return 1
    else: 
        return 0 
parse_value_udf = udf(drive_way, IntegerType())
df = df.withColumn("drive_way", parse_value_udf(df["priority_characteristics"]))


In [0]:
# thêm cột person/agent
def per_or_agent(time_info):
    if "cá nhân" in time_info:
        return "Personal"
    if "môi giới" in time_info:
        return "Agent"
    else: 
        return "Else"
    
parse_value_udf = udf(per_or_agent, StringType())
df = df.withColumn("person/agent", parse_value_udf(df["time_info"]))


In [0]:
# sử lý cột giay to phap ly
def legal_status(value):
    if value is not None:
        if "1" in value:
            return "Đã có sổ"
        if "2" in value:
            return "Đang chờ sổ sổ"
        if "3" in value:
            return "Không có sổ"
        else: 
            return value
    else:
        return "Không có sổ"
parse_value_udf = udf(legal_status, StringType())
df = df.withColumn("legal_status", parse_value_udf(df["legal_status"]))



In [0]:
# sử lý ngày
from pyspark.sql.functions import to_date
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
#df = df.withColumn("date_column", to_date(df["crawl_time"]))
#df = df.withColumn("date_only", to_date(df["crawl_time"].split(" ")[0]))
df = df.withColumn("time_crawled", to_date(df["crawl_time"], "MM/dd/yyyy HH:mm"))

In [0]:
# Ngay đăng bài
from pyspark.sql.functions import expr
from datetime import datetime, timedelta

def date_listed(time_info, time_crawled):
    if "giờ" in time_info:
        return time_crawled
    if "hôm qua" in time_info:
        return time_crawled - timedelta(days=1)
    if "ngày" in time_info:
        match = re.search(r'\d+', time_info)
        if match: 
            days_ago = int(match.group())
            return time_crawled - timedelta(days= days_ago)    
    if "tuần" in time_info:
        return time_crawled - timedelta(days= 8) 
    else: 
        return time_crawled
    
parse_value_udf = udf(date_listed, DateType())
df = df.withColumn("date_listed", parse_value_udf(df["time_info"],df["time_crawled"]))



In [0]:
# Sử lý nội thất nếu null thì là không có
def add_moreinfo(value):
    if value is None:
        return "Không có nội thất"
    else:
         return value
parse_value_udf = udf(add_moreinfo, StringType())
df = df.withColumn("furnishing_status", parse_value_udf(df["furnishing_status"]))



In [0]:
# Sử lý cột land_type
def processland_type(value):
    if value is not None:
        if value == "Đất thổ cư" or value == "Đất nông nghiệp" or value == "Đất nền dự án" or value == "Đất công nghiệp":
            return value
        else: return "Không phải đất"
    else: 
        return "Không phải đất"

parse_value_udf = udf(processland_type, StringType())
df = df.withColumn("land_type", parse_value_udf(df["land_type"]))

In [0]:
def processApartment_status(value):
    if value is not None:
        if "Đã bàn giao" in value or "Chưa bàn giao" in value :
            return value
        else: return "Không phải chung cư"
    else: 
        return "Không phải chung cư"

parse_value_udf = udf(processApartment_status, StringType())
df = df.withColumn("property_status", parse_value_udf(df["property_status"]))

In [0]:
def processApartment_type(value):
    if value is not None:
        if value == "Chung cư" or value == "Duplex" or value == "Penthouse" or value == "Officetel" or value == "Tập thể, cư xá" or value == "Căn hộ dịch vụ, mini":
            return value
        else: return "Không phải chung cư"
    else: 
        return "Không phải chung cư"

parse_value_udf = udf(processApartment_type, StringType())
df = df.withColumn("apartment_type", parse_value_udf(df["apartment_type"]))

### 2.3 Xóa cột dư thừa

In [0]:
columns_to_drop = ["time_info", "crawl_time"]
df = df.drop(*columns_to_drop)

### 2.4 Đổi tên cột

In [0]:
df = df.withColumnRenamed("house_name", "information")
df = df.withColumnRenamed("price", "price_vnd")
df = df.withColumnRenamed("area", "area_m2")
df = df.withColumnRenamed("length", "length_m")
df = df.withColumnRenamed("width", "width_m")
df = df.withColumnRenamed("property_status", "apartment_status")

In [0]:
#display(df)

## 3. Đổ dữ liệu vào lớp silver và kiểm tra

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable("Silver")